In [9]:
%reset -f

In [10]:
import pandas as pd
import numpy as np
import os
import yaml
from easydict import EasyDict

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import normalized_mutual_info_score as NMI

from z_common import config_to_execute
from data_aug.gaussian_blur import GaussianBlur
from clustering import run_clustering
from dataset import get_datasets
from simclr import SimCLR
from log_functions import *

from models.baseline_encoder import Encoder
from models.alexnet_simclr import AlexSimCLR
from models.resnet_simclr import ResNetSimCLR
from loss.nt_xent import NTXentLoss
from functions import *

In [11]:
N_CUDA = 0
PARENT = 'Office31'
DATA_DOMAIN = 'dslr_webcam'
log_dir_opt = f''

config, CONFIG_FILE, log_dir = config_to_execute(DATA_DOMAIN, PARENT, N_CUDA, log_dir_opt)

config.config_path = CONFIG_FILE
config.log_dir = log_dir
config.lap = 1
config.num_lap = 2

==============  dslr_webcam  ==============


In [77]:
import pandas as pd
import numpy as np
import io
pd.set_option('display.max_rows', 100)



log_files = ['../log_dw.txt', '../log_ad.txt', '../log_wa.txt']
domains = ['dw', 'ad', 'wa']
dfs = {}
dfgs = {}
dfg_avgs = {}
for log_file, domain in zip(log_files, domains):
    with open(log_file, 'r') as f:
        text = f.read()
    split_text = text.split('\n')
    log_text = [line for line in split_text if 'nmi:' in line or 'nmi class:' in line or 'domain_accuracy:' in line]
    split_logs = np.array([line.split(':') for line in log_text])

    dft = pd.DataFrame(split_logs, columns=['title', 'result'])

    augs = [
        'none',
        'jigsaw',
        'mask',
        'jigsaw_mask',
        'mask_const_phase',
        'jigsaw_mask_const_phase',
        'jigsaw_const_phase',
    ]
    augs_rep = sum([[aug for i in range(9)] for aug in augs], [])
    df = pd.DataFrame(augs_rep, columns=['augs'])
    df['rep_number'] = sum([[i//3 for i in range(9)] for _ in range(len(augs))], [])
    df['nmi_domain'] = dft[dft['title']=='nmi'].result.values
    df['nmi_class'] = dft[dft['title']=='nmi class'].result.values
    df['domain_accuracy'] = dft[dft['title']=='domain_accuracy'].result.values
    df[['nmi_domain', 'nmi_class', 'domain_accuracy']] = df[['nmi_domain', 'nmi_class', 'domain_accuracy']].astype('float').round(5)
    dfg = df.groupby(['augs', 'rep_number']).mean().reset_index()
    dfg_avg = dfg.drop('rep_number', axis=1).groupby('augs').mean().reset_index()

    dfs[domain] = df
    dfgs[domain] = dfg
    dfg_avgs[domain] = dfg_avg


In [78]:
for df_t, filename in zip([dfgs, dfg_avgs], ['result_dfgs', 'result_dfg_avgs']):
    result_df_dict = {}
    if filename == 'result_dfg':
        col = ['augs', 'rep_number']
    elif filename == 'result_dfg_avgs':
        col = 'augs'

    result_df_dict['title'] = df_t['dw'][col]
    result_df_dict['dslr_webcam'] = df_t['dw'].drop(col, axis=1)
    result_df_dict['amazon_dslr'] = df_t['ad'].drop(col, axis=1)
    result_df_dict['webcam_amazon'] = df_t['wa'].drop(col, axis=1)
    result_df = pd.concat(result_df_dict, axis=1)

    result_df.to_csv(f'../{filename}.csv', header=True, index=False)

In [79]:
result_df

title dslr_webcam                           amazon_dslr  \
                      augs  nmi_domain nmi_class domain_accuracy  nmi_domain   
0                   jigsaw    0.242993  0.385131        0.726477    0.466779   
1       jigsaw_const_phase    0.584586  0.363948        0.918021    0.696833   
2              jigsaw_mask    0.340583  0.363594        0.776403    0.595614   
3  jigsaw_mask_const_phase    0.376274  0.343108        0.792386    0.483602   
4                     mask    0.005430  0.393557        0.527110    0.356069   
5         mask_const_phase    0.078291  0.362040        0.573859    0.240688   
6                     none    0.004156  0.434009        0.522558    0.427743   

                            webcam_amazon                            
  nmi_class domain_accuracy    nmi_domain nmi_class domain_accuracy  
0  0.188594        0.842267      0.400370  0.186771        0.793189  
1  0.184401        0.928674      0.461222  0.198438        0.873999  
2  0.172630        0.888050      0.305454  0.176560        0.767751  
3  0.159416        0.831809      0.270342  0.165554        0.771657  
4  0.186112        0.809050      0.288610  0.192419        0.715454  
5  0.175072        0.641092      0.183101  0.182833        0.663530  
6  0.221882        0.856613      0.262143  0.223847        0.692230